# 🚀 **Week 2 Advanced Model Training - Day 10-11**

## **Multi-Modal Transformer & Advanced Ensemble Training**

This notebook implements the advanced model architectures for Week 2:
- **BERT Sentiment Pipeline** (FinBERT)
- **Multi-Modal Transformer Architecture**
- **Advanced LSTM with Attention**
- **Ensemble System Training**

**Estimated Training Time**: 4-6 hours with GPU

## **1. Environment Setup & Dependencies**

In [ ]:
# Install required dependencies
!pip install transformers torch torchvision torchaudio
!pip install sentence-transformers
!pip install optuna
!pip install lightgbm xgboost
!pip install scikit-learn pandas numpy matplotlib seaborn
!pip install tqdm

In [ ]:
import os  # [FIX]
import random  # [FIX]
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW  # [FIX] use AdamW from torch.optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup  # [FIX]
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    mean_squared_error,
    confusion_matrix,  # [FIX]
    roc_auc_score,  # [FIX]
)
import optuna
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# [FIX] Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## **2. Data Upload & Loading**

**Upload your advanced features dataset** (colab_advanced_features.csv)

In [ ]:
from google.colab import files
import io

# Upload your dataset
print("Please upload your advanced features dataset (colab_advanced_features.csv)")
uploaded = files.upload()

# Load the dataset
for filename in uploaded.keys():
    print(f"Loading {filename}...")
    data = pd.read_csv(io.BytesIO(uploaded[filename]))
    
print(f"Dataset shape: {data.shape}")
print(f"Columns: {len(data.columns)}")
data.head()

## **3. Data Preparation & Feature Engineering**

In [ ]:
def prepare_advanced_features(data):
    """Prepare features for advanced model training"""
    
    # Separate features and targets
    feature_cols = [col for col in data.columns if not any(x in col for x in 
                    ['direction', 'magnitude', 'returns', 'target'])]
    
    # Target variables
    target_cols = [col for col in data.columns if any(x in col for x in 
                   ['direction', 'magnitude'])]
    
    print(f"Feature columns: {len(feature_cols)}")
    print(f"Target columns: {len(target_cols)}")
    
    # Remove any text columns for now (we'll handle BERT separately)
    numeric_features = data[feature_cols].select_dtypes(include=[np.number])
    
    # Handle missing values
    numeric_features = numeric_features.fillna(0)
    
    return numeric_features, target_cols, data

# Prepare features
features, target_cols, full_data = prepare_advanced_features(data)
print(f"\nNumeric features shape: {features.shape}")
print(f"Target columns: {target_cols}")

## **4. BERT Sentiment Analysis Pipeline**

**Task 1**: Train FinBERT for financial sentiment analysis

In [ ]:
class FinancialBERTClassifier(nn.Module):
    def __init__(self, model_name='ProsusAI/finbert', num_classes=3, pooling='cls'):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
        self.pooling = pooling  # [FIX]
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state  # [FIX]
        if self.pooling == 'mean':  # [FIX]
            # Attention-mask aware mean pooling
            mask = attention_mask.unsqueeze(-1).float()
            summed = torch.sum(last_hidden_state * mask, dim=1)
            denom = torch.clamp(mask.sum(dim=1), min=1e-6)
            pooled_output = summed / denom
        else:  # 'cls'
            pooled_output = last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize model and tokenizer
print("Initializing FinBERT model...")
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')  # [FIX]
bert_model = FinancialBERTClassifier(pooling='cls').to(device)  # [FIX]
print("✅ FinBERT model initialized")

In [ ]:
# [FIX] 4-12: Robust FinBERT training with proper tokenization, DataLoader, AMP, scheduler, early stopping, logging
import time  # [FIX]
from dataclasses import dataclass  # [FIX]

# [FIX] Reproducibility
SEED = 42

def set_seed(seed: int = 42):  # [FIX]
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)  # [FIX]

# [FIX] Detect text and label columns
candidate_text_cols = [
    c for c in data.columns
    if data[c].dtype == object and any(k in c.lower() for k in ['text','content','body','title','message','post','headline'])
]
text_col = candidate_text_cols[0] if candidate_text_cols else None
if text_col is None:  # fallback: join all non-target object columns
    non_target_obj = [c for c in data.columns if data[c].dtype == object and not any(x in c for x in ['direction','magnitude','returns','target','label','sentiment'])]
    if len(non_target_obj) > 0:
        text_col = non_target_obj[0]
    else:
        # create synthetic text by joining row values
        text_col = '__auto_text__'
        data[text_col] = data.astype(str).agg(' '.join, axis=1)

label_col_candidates = [c for c in data.columns if any(k in c.lower() for k in ['sentiment','label'])]
if len(label_col_candidates) == 0:
    # fallback to direction columns
    dir_cols = [c for c in data.columns if 'direction' in c.lower()]
    if len(dir_cols) == 0:
        raise ValueError("No label column found. Please include a 'sentiment'/'label' or a '*direction*' column in the dataset.")
    label_col = dir_cols[0]
else:
    label_col = label_col_candidates[0]

# [FIX] Map labels to 3 classes if needed
labels_raw = data[label_col].values
if labels_raw.dtype.kind in {'U', 'S', 'O'}:
    # categorical strings
    unique_vals = sorted(pd.Series(labels_raw).astype(str).unique())
    label2id = {v: i for i, v in enumerate(unique_vals)}
    y_all = np.array([label2id[str(v)] for v in labels_raw])
else:
    # numeric -> map to {-1,0,1} -> {0,1,2}
    vals = pd.Series(labels_raw).astype(float).values
    mapped = np.sign(vals)
    mapped = mapped.astype(int)
    y_all = (mapped + 1).astype(int)
    unique_vals = sorted(np.unique(y_all))
    label2id = {int(v): int(v) for v in unique_vals}

y_num_classes = int(max(y_all)) + 1
id2label = {v: k for k, v in label2id.items()}

print(f"[FIX] Using text column: {text_col}, label column: {label_col}, num_classes={y_num_classes}")

# [FIX] Ensure classifier output matches dataset classes
if getattr(bert_model.classifier, 'out_features', None) != y_num_classes:
    in_features = bert_model.classifier.in_features
    bert_model.classifier = nn.Linear(in_features, y_num_classes).to(device)  # [FIX]

# [FIX] Dataset without tensor conversion
class BERTSentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        return {
            'text': str(self.texts[idx]),
            'label': int(self.labels[idx]),
        }

# [FIX] Collate using tokenizer.pad for dynamic padding
@dataclass
class CollateConfig:
    tokenizer: any
    max_length: int = 128

collate_cfg = CollateConfig(tokenizer=tokenizer, max_length=128)

def collate_fn(batch):  # [FIX]
    texts = [b['text'] for b in batch]
    labels = torch.tensor([b['label'] for b in batch], dtype=torch.long)
    encoded_list = [
        collate_cfg.tokenizer(
            t,
            truncation=True,
            max_length=collate_cfg.max_length,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=False,
        )
        for t in texts
    ]
    enc = collate_cfg.tokenizer.pad(
        encoded_list,
        padding=True,
        max_length=collate_cfg.max_length,
        return_tensors='pt'
    )
    return enc, labels

# [FIX] TimeSeriesSplit without shuffle; ensure no future leakage
tscv = TimeSeriesSplit(n_splits=3)
indices = np.arange(len(data))
splits = list(tscv.split(indices))
train_idx, valid_idx = splits[-1]  # use last split for validation
assert train_idx.max() < valid_idx.min(), "[FIX] Time order violated between train and valid (potential leakage)"  # [FIX]

train_texts = data.iloc[train_idx][text_col].astype(str).tolist()
train_labels = y_all[train_idx]
valid_texts = data.iloc[valid_idx][text_col].astype(str).tolist()
valid_labels = y_all[valid_idx]

train_dataset = BERTSentimentDataset(train_texts, train_labels)
valid_dataset = BERTSentimentDataset(valid_texts, valid_labels)

BATCH_SIZE = 16  # [FIX]

def seed_worker(worker_id):  # [FIX]
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)

generator = torch.Generator()
generator.manual_seed(SEED)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # [FIX] to prevent leakage-like reordering
    num_workers=2,
    collate_fn=collate_fn,
    worker_init_fn=seed_worker,
    generator=generator,
    drop_last=False,
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # [FIX]
    num_workers=2,
    collate_fn=collate_fn,
    worker_init_fn=seed_worker,
    generator=generator,
    drop_last=False,
)

# [FIX] Guard/logging
EPOCHS = 5
print(f"[FIX] epochs={EPOCHS}, len(train_dataset)={len(train_dataset)}, len(train_loader)={len(train_loader)}, len(valid_dataset)={len(valid_dataset)}, len(valid_loader)={len(valid_loader)}")
first_batch_inputs, first_batch_labels = next(iter(train_loader))
for k, v in first_batch_inputs.items():
    print(f"[FIX] first batch {k} shape: {tuple(v.shape)}")
print(f"[FIX] first batch labels shape: {tuple(first_batch_labels.shape)}")

# [FIX] Optimizer param groups with weight decay
no_decay = ["bias", "LayerNorm.weight"]
param_groups = [
    {"params": [p for n, p in bert_model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in bert_model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(param_groups, lr=2e-5)  # [FIX]

# [FIX] Scheduler with warmup
TOTAL_STEPS = max(1, len(train_loader) * EPOCHS)
WARMUP_STEPS = int(0.1 * TOTAL_STEPS)
scheduler = get_linear_schedule_with_warmup(optimizer, WARMUP_STEPS, TOTAL_STEPS)

# [FIX] AMP + GradScaler
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

criterion = nn.CrossEntropyLoss()

# [FIX] EarlyStopping
class EarlyStopping:
    def __init__(self, patience=3, mode='max'):
        self.patience = patience
        self.mode = mode
        self.best = None
        self.counter = 0
        self.best_state = None
        self.best_epoch = -1
    def step(self, value, model, epoch):
        improved = False
        if self.best is None:
            self.best = value
            improved = True
        else:
            if (self.mode == 'max' and value > self.best) or (self.mode == 'min' and value < self.best):
                improved = True
        if improved:
            self.best = value
            self.counter = 0
            self.best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            self.best_epoch = epoch
        else:
            self.counter += 1
        return improved

# [FIX] Training loop with tqdm, AMP, grad clipping, logging
opt_steps = 0
start_time = time.time()
progress_epochs = tqdm(range(EPOCHS), desc='[FIX] Epochs')
best_val_acc = None
best_epoch = -1
scaler_step_count = 0
scaler_update_count = 0

for epoch in progress_epochs:
    bert_model.train()  # [FIX]
    batch_iter = tqdm(train_loader, desc=f"[FIX] Train epoch {epoch}", leave=False)
    for batch in batch_iter:
        inputs, labels = batch
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):  # [FIX]
            logits = bert_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)  # [FIX]
        scaler.step(optimizer)
        scaler.update()
        scaler_step_count += 1
        scaler_update_count += 1
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()
        opt_steps += 1
        batch_iter.set_postfix({"loss": f"{loss.item():.4f}"})

    # Validation
    bert_model.eval()
    val_preds = []
    val_probs = []
    val_labels = []
    with torch.no_grad():  # [FIX] only in eval
        for inputs, labels in tqdm(valid_loader, desc=f"[FIX] Valid epoch {epoch}", leave=False):
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            logits = bert_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            val_probs.append(probs.detach().cpu().numpy())
            val_preds.append(preds.detach().cpu().numpy())
            val_labels.append(labels.detach().cpu().numpy())
    val_probs = np.concatenate(val_probs, axis=0)
    val_preds = np.concatenate(val_preds, axis=0)
    val_labels_np = np.concatenate(val_labels, axis=0)

    val_acc = accuracy_score(val_labels_np, val_preds)

    try:
        if y_num_classes == 2:
            auc = roc_auc_score(val_labels_np, val_probs[:, 1])
        else:
            auc = roc_auc_score(val_labels_np, val_probs, multi_class='ovr', average='macro')
    except Exception:
        auc = None

    progress_epochs.set_postfix({"val_acc": f"{val_acc:.4f}", "auc": f"{auc if auc is not None else 'NA'}"})

    if best_val_acc is None or val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch

    # Early stopping
    # Using separate early stopper to keep best_state
    if epoch == 0:
        early_stopper = EarlyStopping(patience=3, mode='max')
    early_stopper.step(val_acc, bert_model, epoch)
    if early_stopper.counter >= early_stopper.patience:
        print(f"[FIX] Early stopping triggered at epoch {epoch}")
        break

train_time_sec = time.time() - start_time

print(f"[FIX] optimizer steps = {opt_steps}")
print(f"[FIX] optimizer steps >= len(train_loader) ? {opt_steps >= len(train_loader)} (len(train_loader)={len(train_loader)})")  # [FIX]
print(f"[FIX] scaler steps = {scaler_step_count}, scaler updates = {scaler_update_count}")

# [FIX] Assertions
assert EPOCHS > 0 and len(train_loader) > 0 and opt_steps > 0 and opt_steps >= len(train_loader)

# [FIX] Evaluate pooling option 'mean' vs 'cls' on validation (no retraining)
original_pooling = bert_model.pooling if hasattr(bert_model, 'pooling') else 'cls'
bert_model.pooling = 'mean'
bert_model.eval()
val_preds_mean = []
with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs = {k: v.to(device) for k, v in inputs.items()}
        logits = bert_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        val_preds_mean.append(preds)
val_preds_mean = np.concatenate(val_preds_mean)
val_acc_mean = accuracy_score(val_labels_np, val_preds_mean)
bert_model.pooling = original_pooling

best_pooling = 'mean' if val_acc_mean > best_val_acc else original_pooling
best_pool_val = max(best_val_acc, val_acc_mean)
assert 0.0 <= best_pool_val <= 1.0  # [FIX]
print(f"[FIX] Pooling comparison -> cls_acc={best_val_acc:.4f}, mean_acc={val_acc_mean:.4f}; best='{best_pooling}'")

# [FIX] Reports
print("[FIX] Classification report (validation):")
print(classification_report(val_labels_np, val_preds, zero_division=0))
try:
    cm = confusion_matrix(val_labels_np, val_preds)
    print("[FIX] Confusion matrix:\n", cm)
except Exception:
    pass

# [FIX] Save best checkpoint
best_state = early_stopper.best_state if early_stopper.best_state is not None else bert_model.state_dict()
bert_model.load_state_dict(best_state)

hyperparams = {
    "lr": 2e-5,
    "weight_decay": 0.01,
    "batch_size": BATCH_SIZE,
    "max_len": collate_cfg.max_length,
    "epochs": EPOCHS,
    "warmup_ratio": 0.1,
    "pooling": best_pooling,  # [FIX] save best pooling
    "seed": SEED,
}

ckpt = {
    "state_dict": best_state,
    "model_name": "ProsusAI/finbert",
    "label2id": label2id,
    "id2label": id2label,
    "hyperparams": hyperparams,
    "best_val": float(best_pool_val),
    "best_epoch": int(best_epoch),
}

torch.save(ckpt, "finbert_week2.pt")  # [FIX]
try:
    tokenizer.save_pretrained("./tokenizer_week2")  # [FIX]
except Exception as e:
    print(f"[FIX] tokenizer save warning: {e}")

# [FIX] Prepare metrics for aggregation
finbert_metrics = {
    'Model': 'FinBERT',
    'Accuracy': float(best_pool_val),
    'Type': 'NLP',
    'val_metric': float(best_pool_val),
    'train_time_sec': float(train_time_sec),
    'params': sum(p.numel() for p in bert_model.parameters()),
    'opt_steps': int(opt_steps),
}
print(f"[FIX] FinBERT best_val={best_pool_val:.4f} at epoch={best_epoch}")


## **5. Multi-Modal Transformer Architecture**

**Task 2**: Train transformer model for temporal sequence prediction

In [ ]:
class MultiModalTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_heads=8, num_layers=6, num_classes=2):
        super().__init__()
        
        self.input_projection = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = nn.Parameter(torch.randn(1000, hidden_dim))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x = self.input_projection(x)
        
        # Add positional encoding
        seq_len = x.size(1)
        x = x + self.positional_encoding[:seq_len].unsqueeze(0)
        
        # Transformer encoding
        x = self.transformer(x)
        
        # Global average pooling
        x = x.mean(dim=1)
        
        # Classification
        logits = self.classifier(x)
        return logits

# Initialize transformer
transformer_model = MultiModalTransformer(
    input_dim=features.shape[1],
    hidden_dim=256,
    num_heads=8,
    num_layers=6
).to(device)

print(f"✅ Multi-modal transformer initialized with {features.shape[1]} input features")

## **6. Advanced LSTM with Attention**

**Task 3**: Train bidirectional LSTM with attention mechanism

In [ ]:
class AttentionLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, num_classes=2):
        super().__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=0.1
        )
        
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,  # bidirectional
            num_heads=4,
            batch_first=True
        )
        
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)
        
    def forward(self, x):
        # LSTM processing
        lstm_out, _ = self.lstm(x)
        
        # Self-attention
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Global average pooling
        pooled = attn_out.mean(dim=1)
        
        # Classification
        logits = self.classifier(pooled)
        return logits

# Initialize LSTM
lstm_model = AttentionLSTM(
    input_dim=features.shape[1],
    hidden_dim=128,
    num_layers=2
).to(device)

print(f"✅ Attention LSTM initialized with {features.shape[1]} input features")

## **7. Model Training Pipeline**

In [ ]:
def train_advanced_models(features, data, target_col='GME_direction_1d'):
    """Train all advanced models"""
    
    # Prepare data
    X = features.values
    y = data[target_col].values
    
    # Time series split
    tscv = TimeSeriesSplit(n_splits=3)
    
    models = {
        'transformer': transformer_model,
        'lstm': lstm_model
    }
    
    results = {}
    
    for model_name, model in models.items():
        print(f"\n🔄 Training {model_name.upper()} model...")
        
        # Convert to tensors
        X_tensor = torch.FloatTensor(X).to(device)
        y_tensor = torch.LongTensor(y).to(device)
        
        # Training setup
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        # [FIX] Track steps and time
        opt_steps = 0
        start_time = time.time()
        
        # Training loop
        model.train()  # [FIX]
        for epoch in range(5):
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(X_tensor.unsqueeze(1))  # Add sequence dimension
            loss = criterion(outputs, y_tensor)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # [FIX]
            optimizer.step()
            opt_steps += 1  # [FIX]
            
            if epoch % 2 == 0:
                print(f"  [FIX] {model_name} epoch {epoch}: loss={loss.item():.4f}")
        
        train_time_sec = time.time() - start_time  # [FIX]
        
        # Evaluation
        model.eval()
        with torch.no_grad():
            predictions = model(X_tensor.unsqueeze(1))
            pred_labels = torch.argmax(predictions, dim=1).cpu().numpy()
            accuracy = accuracy_score(y, pred_labels)
            
        results[model_name] = {
            'accuracy': accuracy,
            'model': model,
            'opt_steps': opt_steps,  # [FIX]
            'train_time_sec': train_time_sec,  # [FIX]
            'params': sum(p.numel() for p in model.parameters()),  # [FIX]
        }
        
        print(f"  ✅ {model_name.upper()} Accuracy: {accuracy:.4f}")
    
    return results

# Train all models
print("\n🚀 Starting advanced model training...")
training_results = train_advanced_models(features, data)
print("\n🎉 Advanced model training completed!")

## **8. Ensemble System Training**

**Task 4**: Create and train ensemble system

In [ ]:
class AdvancedEnsemble:
    def __init__(self, models, weights=None):
        self.models = models
        self.weights = weights if weights else [1/len(models)] * len(models)
        
    def predict(self, X):
        predictions = []
        
        for model_name, model_info in self.models.items():
            model = model_info['model']
            model.eval()
            
            with torch.no_grad():
                X_tensor = torch.FloatTensor(X).to(device)
                outputs = model(X_tensor.unsqueeze(1))
                pred_probs = torch.softmax(outputs, dim=1).cpu().numpy()
                predictions.append(pred_probs)
        
        # Weighted ensemble
        ensemble_pred = np.zeros_like(predictions[0])
        for i, (pred, weight) in enumerate(zip(predictions, self.weights)):
            ensemble_pred += weight * pred
        
        return np.argmax(ensemble_pred, axis=1)

# [FIX] Include FinBERT into ensemble inputs as optional
if 'finbert_metrics' in globals():
    print("[FIX] FinBERT metrics available; will include its score separately in results aggregation.")

# Create ensemble
ensemble = AdvancedEnsemble(training_results)

# Evaluate ensemble
X_test = features.values
y_test = data['GME_direction_1d'].values

ensemble_preds = ensemble.predict(X_test)
ensemble_accuracy = accuracy_score(y_test, ensemble_preds)

# [FIX] Add per-model metrics row for results CSV
for mname, minfo in training_results.items():
    training_results[mname]['val_metric'] = minfo['accuracy']  # [FIX]

print(f"\n🎯 Ensemble Accuracy: {ensemble_accuracy:.4f}")
print("✅ Advanced ensemble system completed!")

## **9. Results Summary & Model Comparison**

In [ ]:
# Compile results
results_summary = {
    'Model': [],
    'Accuracy': [],
    'Type': []
}

# [FIX] Add FinBERT result first
if 'finbert_metrics' in globals():
    results_summary['Model'].append(finbert_metrics['Model'])
    results_summary['Accuracy'].append(finbert_metrics['Accuracy'])
    results_summary['Type'].append(finbert_metrics['Type'])

# Add individual model results
for model_name, result in training_results.items():
    results_summary['Model'].append(model_name.title())
    results_summary['Accuracy'].append(result['accuracy'])
    results_summary['Type'].append('Individual')

# Add ensemble result
results_summary['Model'].append('Ensemble')
results_summary['Accuracy'].append(ensemble_accuracy)
results_summary['Type'].append('Ensemble')

# Create results DataFrame
results_df = pd.DataFrame(results_summary)
print("\n📊 Week 2 Advanced Model Results:")
print(results_df)

# [FIX] Save extended CSV with more metrics if available
extended_rows = []
if 'finbert_metrics' in globals():
    extended_rows.append({
        'Model': finbert_metrics['Model'],
        'Accuracy': finbert_metrics['Accuracy'],
        'Type': finbert_metrics['Type'],
        'val_metric': finbert_metrics['val_metric'],
        'train_time_sec': finbert_metrics['train_time_sec'],
        'params': finbert_metrics['params'],
        'opt_steps': finbert_metrics['opt_steps'],
    })
for mname, minfo in training_results.items():
    extended_rows.append({
        'Model': mname.title(),
        'Accuracy': minfo.get('accuracy', None),
        'Type': 'Individual',
        'val_metric': minfo.get('val_metric', None),
        'train_time_sec': minfo.get('train_time_sec', None),
        'params': minfo.get('params', None),
        'opt_steps': minfo.get('opt_steps', None),
    })
if extended_rows:
    extended_df = pd.DataFrame(extended_rows)
    extended_df.to_csv('week2_advanced_results_extended.csv', index=False)
    print("[FIX] Saved extended results to week2_advanced_results_extended.csv")

# Plot results
plt.figure(figsize=(10, 6))
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']
bars = plt.bar(results_df['Model'], results_df['Accuracy'], color=colors)
plt.title('Week 2 Advanced Model Performance', fontsize=16, fontweight='bold')
plt.ylabel('Accuracy')
plt.ylim(0, 1)

# Add value labels on bars
for bar, acc in zip(bars, results_df['Accuracy']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎉 Week 2 Advanced Model Training Complete!")
print("\n📋 Next Steps:")
print("1. Save trained models")
print("2. Compare with Week 1 baseline")
print("3. Perform statistical validation")
print("4. Conduct ablation studies")

## **10. Save Trained Models**

In [ ]:
# [FIX] 변경 요약 출력 셀: 노트북 전체에서 '# [FIX]'가 포함된 줄을 수집해 라인 번호와 함께 출력
import json, re, sys
from pathlib import Path

nb_path = Path("colab_advanced_model_training.ipynb")
try:
    import nbformat
    nb = nbformat.read(nb_path.open('r'), as_version=4)
    fix_lines = []
    for ci, cell in enumerate(nb.cells):
        if cell.cell_type != 'code':
            continue
        lines = cell.source.splitlines()
        for li, line in enumerate(lines, start=1):
            if '# [FIX]' in line:
                fix_lines.append({
                    'cell_index': ci,
                    'line_in_cell': li,
                    'text': line.strip(),
                })
    print("변경 요약 (셀 인덱스/셀 내부 라인):")
    for item in fix_lines:
        print(f" - cell {item['cell_index']} line {item['line_in_cell']}: {item['text']}")
    print(f"총 {len(fix_lines)}개 변경 라인 탐지")
except Exception as e:
    print(f"변경 요약 수집 실패: {e}")


In [ ]:
# Save trained models
import pickle

# [FIX] Ensure opt_steps asserted before saving
if 'finbert_metrics' in globals():
    assert finbert_metrics['opt_steps'] > 0  # [FIX]

# [FIX] Save FinBERT artifact already handled above; extend ensemble/meta saving

def save_models(training_results, ensemble, results_df):
    """Save all trained models and results"""
    
    # Save individual models
    for model_name, result in training_results.items():
        torch.save(result['model'].state_dict(), f'{model_name}_week2.pth')
        print(f"✅ Saved {model_name} model")
    
    # Save ensemble with meta  # [FIX]
    meta = {
        'component_models': list(training_results.keys()),
        'weights': None,
        'created_at': time.strftime('%Y-%m-%d %H:%M:%S'),
    }
    with open('ensemble_week2.pkl', 'wb') as f:
        pickle.dump({'ensemble': ensemble, 'meta': meta}, f)  # [FIX]
    print("✅ Saved ensemble model with meta")
    
    # Save results
    results_df.to_csv('week2_advanced_results.csv', index=False)
    print("✅ Saved results summary")
    
    # Create download links
    from google.colab import files
    
    print("\n📥 Download trained models:")
    for model_name in training_results.keys():
        files.download(f'{model_name}_week2.pth')
    
    files.download('ensemble_week2.pkl')
    files.download('week2_advanced_results.csv')
    if os.path.exists('week2_advanced_results_extended.csv'):
        files.download('week2_advanced_results_extended.csv')  # [FIX]

# Save all models
save_models(training_results, ensemble, results_df)

print("\n🎯 Week 2 Advanced Model Training Successfully Completed!")
print("\n📊 Key Achievements:")
print(f"- Trained {len(training_results)} advanced models")
print(f"- Created ensemble system")
print(f"- Best accuracy: {results_df['Accuracy'].max():.4f}")
print(f"- Ensemble accuracy: {ensemble_accuracy:.4f}")